# Outflow Rate
This notebook calculates the mass and energy outflow rate of a given simulation <br>
The mass outflow rate is calculated using the equation found in: https://arxiv.org/pdf/1902.05554 <br>
We use a similar approach to calculate the energy outflow rate https://arxiv.org/pdf/2002.10468: $\dot{E} = \Sigma (E v_r)/\Delta r$ for $E = \rho(1/2 v^2 + e_{th}$
## Utilities

In [ ]:
# Loading libraries and key coordinates
import glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.legend_handler import HandlerTuple
from scipy import interpolate
from matplotlib.lines import Line2D
from scipy import stats
import seaborn as sns
from scipy import optimize
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FuncFormatter

# Utilities file
from utilities import Temp_S
 
################################################
##### PHYSICAL CONSTANTS #####
gamma = 5/3
kb = 1.3807e-16 # Boltzmann Constant in CGS or erg/K
seconds_in_myrs = 3.15576e+13
seconds_in_yrs = 3.154e+7
sb_constant_cgs = 5.670374419e-5 # ergs/cm^2/K^4/s
T_COLD_MAX = 3e4
from utilities import HYDROGEN_MASS_FRACTION # 0.76
from utilities import PROTON_MASS_GRAMS # 1.67262192e-24 
from utilities import GRAVITIONAL_CONSTANT_IN_CGS #  6.6738e-8
from utilities import HUBBLE #  3.2407789e-18
from utilities import M_PI #  3.14159265358979323846

plt.style.use("seaborn-paper")
##### UNITS #####
UnitVelocity_in_cm_per_s = 1e5 # 10 km/sec  
UnitLength_in_cm = 3.085678e21 # 1 kpc
UnitMass_in_g  = 1.989e33 # 1 solar mass
UnitTime_in_s = UnitLength_in_cm / UnitVelocity_in_cm_per_s # 3.08568e+16 seconds 
UnitEnergy_in_cgs = UnitMass_in_g * pow(UnitLength_in_cm, 2) / pow(UnitTime_in_s, 2) # 1.9889999999999999e+43 erg
UnitDensity_in_cgs = UnitMass_in_g / pow(UnitLength_in_cm, 3) # 6.76989801444063e-32 g/cm^3
UnitPressure_in_cgs = UnitMass_in_g / UnitLength_in_cm / pow(UnitTime_in_s, 2) # 6.769911178294542e-22 barye
UnitNumberDensity = UnitDensity_in_cgs/PROTON_MASS_GRAMS
UnitEnergyDensity = UnitEnergy_in_cgs/pow(UnitLength_in_cm, 3)

G_code = GRAVITIONAL_CONSTANT_IN_CGS/(pow(UnitLength_in_cm,3) * pow(UnitMass_in_g, -1) * pow(UnitTime_in_s, -2)) 
Hubble_code = HUBBLE * UnitTime_in_s # All.Hubble in Arepo


###### KEY SIMULATION PARAMETERS ######
boxsize = 100
midpoint = boxsize/2 
center_boxsize = 10
center_boxsize_large = 15 
cells_per_dim = 301 
cells_per_dim_large = 451 # for the MW plots
deviation = 10
# deviation_large = 7.5
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2  + deviation # boundary of histogram - upper bound

angle_l = 60
dx = center_boxsize/300
eps = dx/1e6
Z_solar = 0.02


def custom_tick_labels(x, pos):
    return f"{x - boxsize/2:.0f}"

# Loading Parameters
LOAD = True
outputs = ["./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_high_alpha/", "./outflow_cooling/output_high_beta/", "./outflow_cooling/output_high_alpha_high_beta/"]
labeling = []

# Periodic Starbursts
# LOAD = False
# outputs = ["./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_long_bursts/", "./outflow_cooling/output_short_bursts/", "./outflow_cooling/output_short_fast_bursts/"]
# labeling = [r"Continuous", r"$\rm t_{burst} = 5 \, Myr,\,\, t_{rest} =  10 \, Myr$", 
        #   r"$\rm t_{burst} = 1 \, Myr,\,\, t_{rest} =  5 \, Myr$", r"$\rm t_{burst} = 1 \, Myr,\,\, t_{rest} =  1 \, Myr$"]
times_snap = []
###############################################################
###############################################################
# labeling = ["M82/LMC", "MW", "SMC"]

if len(outputs) == 2:
    coloring = ["crimson", "teal"]
elif len(outputs) == 3:
    coloring = ["crimson", "teal", "darkgreen"]
else:
    color_map = plt.get_cmap('turbo')
    coloring = color_map(np.linspace(0, 1, len(outputs)))

n_snaps = 4 
first = 100 - n_snaps 
# first = 70
last = 100
step = 1


## Get the Outflow Rates

In [ ]:

def calculate_std(flux_snaps): 
    return np.percentile(flux_snaps, 16, axis=0), np.percentile(flux_snaps, 84, axis=0)   # shape: (len(r_analysis), n_bins)

def plot_fluxes(ax, x_axis, fluxes, std_b, std_a, labeling, coloring,lining, shading):
    ax.plot(x_axis, fluxes, label=labeling, linestyle=lining, color=coloring)
    if shading: ax.fill_between(x_axis, std_b, std_a, color=coloring,  alpha=0.25)

n_bins = 301
theta = 60
n_snaps = 4
r_analysis = [5, 10, 20] 

r_bins = np.linspace(0.0, 30, n_bins+1)
dr = np.diff(r_bins)[0]
t_bins = np.linspace(3.5, 8, n_bins+1)
nd_bins = np.linspace(-3, 1, n_bins+1)
rv_bins = np.linspace(0, 2000, n_bins+1)
flux_statistics = {
    "mradial_flux_radius": np.zeros(shape=(len(outputs), n_bins)),
    "mradial_flux_radius_cold": np.zeros(shape=(len(outputs), n_bins)),

    "eradial_flux_radius": np.zeros(shape=(len(outputs), n_bins)),
    "eradial_flux_radius_cold": np.zeros(shape=(len(outputs), n_bins)),
    "keradial_flux_radius" : np.zeros(shape=(len(outputs), n_bins)),
    "ieradial_flux_radius" : np.zeros(shape=(len(outputs), n_bins)),

    "momradial_flux_radius": np.zeros(shape=(len(outputs), n_bins)),
    "momradial_flux_radius_cold": np.zeros(shape=(len(outputs), n_bins)),

    "rad_vels": np.zeros(shape=(len(outputs), n_bins)),

    "m_flux_84" : np.zeros(shape=(len(outputs), n_bins)),
    "m_flux_84_cold" : np.zeros(shape=(len(outputs), n_bins)),
    "m_flux_84_hot" : np.zeros(shape=(len(outputs), n_bins)),

    "m_flux_16" : np.zeros(shape=(len(outputs), n_bins)),
    "m_flux_16_cold" : np.zeros(shape=(len(outputs), n_bins)),
    "m_flux_16_hot" : np.zeros(shape=(len(outputs), n_bins)),

    "e_flux_84" : np.zeros(shape=(len(outputs), n_bins)),
    "e_flux_84_cold" : np.zeros(shape=(len(outputs), n_bins)),
    "e_flux_84_hot" : np.zeros(shape=(len(outputs), n_bins)),

    "e_flux_16" : np.zeros(shape=(len(outputs), n_bins)),
    "e_flux_16_cold" : np.zeros(shape=(len(outputs), n_bins)),
    "e_flux_16_hot" : np.zeros(shape=(len(outputs), n_bins)),

    "mom_flux_84" : np.zeros(shape=(len(outputs), n_bins)),
    "mom_flux_84_cold" : np.zeros(shape=(len(outputs), n_bins)),
    "mom_flux_84_hot" : np.zeros(shape=(len(outputs), n_bins)),

    "mom_flux_16" : np.zeros(shape=(len(outputs), n_bins)),
    "mom_flux_16_cold" : np.zeros(shape=(len(outputs), n_bins)),
    "mom_flux_16_hot" : np.zeros(shape=(len(outputs), n_bins)),
}
m_radial_snapshots = []
e_radial_snapshots = []
mom_radial_snapshots = []
alphas = []
betas = []
tv_flux = np.zeros(shape=(len(outputs), n_bins, n_bins)) 

data_init = {}

filename = outputs[0] + "snap_000.hdf5"
with h5py.File(filename,'r') as f:
    for key in f['PartType0']:
        data_init[key] = f['PartType0'][key][()]
x_coord_0 = data_init["Coordinates"][:,0] 
y_coord_0 = data_init["Coordinates"][:,1]
z_coord_0 = data_init["Coordinates"][:,2]
abundance_0 = data_init["ElectronAbundance"]
number_density_0 = data_init["Density"]*UnitNumberDensity
internal_energy_0 = data_init["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
temperature_0 = Temp_S(abundance_0, internal_energy_0)
rad_x0, rad_y0, rad_z0 = x_coord_0 - 0.5*boxsize, y_coord_0 - 0.5*boxsize, z_coord_0 - 0.5*boxsize
nd_h = np.percentile(number_density_0[rad_z0 >= 2], 99.85)
t_l = np.percentile(temperature_0[rad_z0 >= 2], 0.15)

for o, output in enumerate(outputs): 
# adapts the temperature ranges used in CGOLs
    rm_flux_snapshots = []
    rm_flux_snapshots_cold = [] 

    re_flux_snapshots = []
    re_flux_snapshots_cold = [] 
    ke_flux_snapshots = []
    ie_flux_snapshots = []

    mom_flux_snapshots = []
    mom_flux_snapshots_cold = []
    rad_vels = []

    data = {}
    for i in np.arange(first, last + step, step):
        filename = output + "snap_%03d.hdf5" % i
        print(filename)
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)
        boxsize = parameters["BoxSize"]
        t = header["Time"]
        if (i == first) or (i == last): print("times = %0.5f Myr" % t)
        alpha = parameters["E_load"]
        beta = parameters["M_load"]
        R = parameters["injection_radius"]
        coord = data["Coordinates"]
        x_coord = data["Coordinates"][:,0] 
        y_coord = data["Coordinates"][:,1]
        z_coord = data["Coordinates"][:,2]
        density = data["Density"]
        internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the internal energy
        masses = data["Masses"] 
    
        # pressures = data["Pressure"] 
        abundance = data["ElectronAbundance"]
        temperature = Temp_S(abundance, internal_energy)
        number_density = density*UnitNumberDensity # Units: cm^{-3}

        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1] 
        vel_z = data["Velocities"][:,2] 
        vel = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
        E_internal = internal_energy*masses # NOTE: This is the actual internal energy
        R = parameters["injection_radius"]
        sfr = parameters["sfr"]
        rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
        radius = np.sqrt(rad_x**2 + rad_y**2 + rad_z**2)
        radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s
        theta = np.arccos(np.abs(rad_z)/(radius))*180/np.pi 
        alphas.append(alpha)
        betas.append(beta)
        momentum = masses*radial_velocity


        bg_cells = (number_density <= nd_h) & (temperature >= t_l) & (np.abs(radial_velocity) <= 40) # Gets rids of most BG cells.

        angular_region = (np.abs(theta) <= angle_l) & (radius <= 50) # & (np.abs(rad_z) >= R)# & (radial_velocity >= 0)# Excludes anything with absolute angles greater than 60 
        angular_bg_mask = (angular_region) & (~bg_cells)

        angular_region_cold = (angular_region) & (temperature <= T_COLD_MAX) 
        angular_bg_mask_cold = (angular_bg_mask) & (temperature <= T_COLD_MAX) 

        # Mass outflow rates
        mass_tot, m_binned, _ = stats.binned_statistic(radius[angular_region], masses[angular_region]*radial_velocity[angular_region], bins=r_bins, statistic="sum")
        mass_C, m_binned_C, _ = stats.binned_statistic(radius[angular_region_cold], masses[angular_region_cold]*radial_velocity[angular_region_cold], bins=r_bins, statistic="sum")

        mass_flux_radius = mass_tot/dr/(UnitTime_in_s/seconds_in_yrs) 
        mass_flux_radius_cold = mass_C/dr/(UnitTime_in_s/seconds_in_yrs) 

        m_center = 0.5 * (m_binned[1:] + m_binned[:-1])

        rm_flux_snapshots.append(mass_flux_radius)
        rm_flux_snapshots_cold.append(mass_flux_radius_cold) # cold = 2e4K 
  
        flux_statistics["mradial_flux_radius"][o] += mass_flux_radius
        flux_statistics["mradial_flux_radius_cold"][o] += mass_flux_radius_cold



        # Energy outflow rates9
        kinetic_e = (1/2*masses*pow(vel,2))
        internal_e = (internal_energy*masses)
        total_energy = kinetic_e + internal_e

        E_tot, e_binned, _ = stats.binned_statistic(radius[angular_bg_mask], total_energy[angular_bg_mask]*radial_velocity[angular_bg_mask], bins=r_bins, statistic="sum")
        E_C, e_binned_C, _ = stats.binned_statistic(radius[angular_bg_mask_cold], total_energy[angular_bg_mask_cold]*radial_velocity[angular_bg_mask_cold], bins=r_bins, statistic="sum")

        E_flux_radius = E_tot/dr/(UnitTime_in_s)*UnitEnergy_in_cgs  # ergs/second
        E_flux_radius_cold = E_C/dr/(UnitTime_in_s)*UnitEnergy_in_cgs  # ergs/second

        e_center = 0.5 * (e_binned[1:] + e_binned[:-1])

        ## Kinetic Energy
        E_kin, e_binned, _ = stats.binned_statistic(radius[angular_bg_mask], kinetic_e[angular_bg_mask]*radial_velocity[angular_bg_mask], bins=r_bins, statistic="sum")
        E_kin_C, e_binned_C, _ = stats.binned_statistic(radius[angular_bg_mask_cold], kinetic_e[angular_bg_mask_cold]*radial_velocity[angular_bg_mask_cold], bins=r_bins, statistic="sum")
        KE_flux_radius = E_kin/dr/(UnitTime_in_s)*UnitEnergy_in_cgs#  * UnitEnergy_in_cgs # ergs/second
        print(KE_flux_radius[e_binned[:-1] <= 0.5])
        ## Internal Energy
        E_in, e_binned, _ = stats.binned_statistic(radius[angular_bg_mask], internal_e[angular_bg_mask]*radial_velocity[angular_bg_mask], bins=r_bins, statistic="sum")
        E_in_C, e_binned_C, _ = stats.binned_statistic(radius[angular_bg_mask_cold], internal_e[angular_bg_mask_cold]*radial_velocity[angular_bg_mask_cold], bins=r_bins, statistic="sum")
        IE_flux_radius = E_in/dr/(UnitTime_in_s)*UnitEnergy_in_cgs # ergs/second

        re_flux_snapshots.append(E_flux_radius)
        re_flux_snapshots_cold.append(E_flux_radius_cold) # cold = 2e4K 

        flux_statistics["eradial_flux_radius"][o] += E_flux_radius
        flux_statistics["eradial_flux_radius_cold"][o] += E_flux_radius_cold

        flux_statistics["keradial_flux_radius"][o] += KE_flux_radius
        flux_statistics["ieradial_flux_radius"][o] += IE_flux_radius

        # Momentum outflow rates - includes on the kinetic
        mom_tot, p_binned, _ = stats.binned_statistic(radius[angular_bg_mask], momentum[angular_bg_mask]*radial_velocity[angular_bg_mask], bins=r_bins, statistic="sum")
        mom_C, p_binned_C, _ = stats.binned_statistic(radius[angular_bg_mask_cold], momentum[angular_bg_mask_cold]*radial_velocity[angular_bg_mask_cold], bins=r_bins, statistic="sum")

        mom_flux_radius = mom_tot/dr/(UnitTime_in_s/seconds_in_yrs) # so solar masses * km/s /year
        mom_flux_radius_cold = mom_C/dr/(UnitTime_in_s/seconds_in_yrs) 

        p_bins_center = 0.5 * (p_binned[1:] + p_binned[:-1])

        mom_flux_snapshots.append(mom_flux_radius)
        mom_flux_snapshots_cold.append(mom_flux_radius_cold) # cold = 2e4K 
  
        flux_statistics["momradial_flux_radius"][o] += mom_flux_radius
        flux_statistics["momradial_flux_radius_cold"][o] += mom_flux_radius_cold
        
        rvs, rv_bins, _ =  stats.binned_statistic(radius[angular_bg_mask], radial_velocity[angular_bg_mask], bins=r_bins, statistic="median")
        flux_statistics["rad_vels"][o] += rvs

    if LOAD: labeling.append(r"$\alpha = %.1f, \beta = %.1f$" % (parameters["E_load"], parameters["M_load"]))
    m_radial_snapshots.append(m_binned[:-1]) 
    e_radial_snapshots.append(e_binned[:-1]) 
    mom_radial_snapshots.append(p_binned[:-1])

    flux_statistics["m_flux_16"][o], flux_statistics["m_flux_84"][o] = calculate_std(rm_flux_snapshots)
    # flux_statistics["m_flux_16_cold"][o], flux_statistics["m_flux_84_cold"][o] = calculate_std(rm_flux_snapshots_cold)

    flux_statistics["e_flux_16"][o], flux_statistics["e_flux_84"][o] = calculate_std(re_flux_snapshots)
    # flux_statistics["e_flux_16_cold"][o], flux_statistics["e_flux_84_cold"][o] = calculate_std(re_flux_snapshots_cold)

    flux_statistics["mom_flux_16"][o], flux_statistics["mom_flux_84"][o] = calculate_std(mom_flux_snapshots)
    # flux_statistics["e_flux_16_cold"][o], flux_statistics["e_flux_84_cold"][o] = calculate_std(re_flux_snapshots_cold)



print(len(np.arange(first, last + step, step)))
print((last - first)/step + 1)

for key in list(flux_statistics.keys())[:9]:
    flux_statistics[key] /= (last - first)/step + 1 # 100 - 97 + 1 = 4 
    print(flux_statistics[key])

In [ ]:

print(len(np.arange(first, last + step, step)))
print((last - first)/step + 1)

# labeling = [r"$\alpha=0.9, \beta=0.6$", r"$\alpha=1.8,\beta=0.6$", r"$\alpha=0.9,\beta=1.0$", r"$\alpha=1.8, \beta=1.0$"]
rflux_element =  [Line2D([0], [0], marker='o', color=coloring[0]),
                   Line2D([0], [0], marker='o', color=coloring[1]),
                   Line2D([0], [0], marker='o', color=coloring[2])]

element_labels = [labeling[0], labeling[1], labeling[2], labeling[3]] #   r"$r = 1 \, \, kpc$", r"$r = 3 \, \, kpc$"]

fig = plt.figure(figsize=(14, 12))
fig.set_rasterized(True)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

# ax2 = fig.add_subplot(gs1[2, 0])
# ax3 = fig.add_subplot(gs1[2, 1])
# ax4 = fig.add_subplot(gs1[2, 2])

tcat_labels= [
                Line2D([0], [0], color='black', linestyle='solid'),        
                Line2D([0], [0], color='black', linestyle='dashed'),
            ]
tcat_names = ["Total", r"$T \leq 1e4.5 \, K$ "] 
              
            #   "$5.5e5 \, K \,\, \geq T \,\, > 1e4.5 \, K$ ", "$T > 5e5 \, K$ "]

em_labels = [
                Line2D([0], [0], color='black', linestyle='solid'),        
                Line2D([0], [0], color='black', linestyle='dotted'),        
                Line2D([0], [0], color='black', linestyle='dashdot'),  
            ]
em_names = ["Total", "Kinetic", "Internal"] 

# to get the momentum outflow in cgs
P_flux_cgs = flux_statistics["momradial_flux_radius"]*UnitMass_in_g*UnitVelocity_in_cm_per_s/seconds_in_yrs
P_flux_cgs_cold = flux_statistics["momradial_flux_radius_cold"]*UnitMass_in_g*UnitVelocity_in_cm_per_s/seconds_in_yrs


print(r_analysis)
for i in np.arange(len(outputs)):
    print(np.unique(e_radial_snapshots))
    print(outputs[i])
    print(labeling[i])
    plot_fluxes(ax1, m_radial_snapshots[i], flux_statistics["mradial_flux_radius"][i], flux_statistics["m_flux_16"][i], flux_statistics["m_flux_84"][i], labeling[i], coloring[i], "solid", True)
    plot_fluxes(ax1, m_radial_snapshots[i], flux_statistics["mradial_flux_radius_cold"][i], flux_statistics["m_flux_16_cold"][i], flux_statistics["m_flux_84_cold"][i], None, coloring[i], "dashed", False)
    plot_fluxes(ax2, e_radial_snapshots[i], flux_statistics["eradial_flux_radius"][i], flux_statistics["e_flux_16"][i], flux_statistics["e_flux_84"][i], labeling[i], coloring[i], "solid", False)
    # plot_fluxes(ax2, e_radial_snapshots[i], flux_statistics["eradial_flux_radius_cold"][i], flux_statistics["e_flux_16_cold"][i], flux_statistics["e_flux_84_cold"][i], None, coloring[i], "dashed", False)
    plot_fluxes(ax2, e_radial_snapshots[i], flux_statistics["keradial_flux_radius"][i], flux_statistics["e_flux_16_cold"][i], flux_statistics["e_flux_84_cold"][i], None, coloring[i], "dotted", False)
    plot_fluxes(ax2, e_radial_snapshots[i], flux_statistics["ieradial_flux_radius"][i], flux_statistics["e_flux_16_cold"][i], flux_statistics["e_flux_84_cold"][i], None, coloring[i], "dashdot", False)

    plot_fluxes(ax3, mom_radial_snapshots[i], flux_statistics["momradial_flux_radius"][i], flux_statistics["mom_flux_16"][i], flux_statistics["mom_flux_84"][i], labeling[i], coloring[i], "solid", True)
    plot_fluxes(ax3, mom_radial_snapshots[i], flux_statistics["momradial_flux_radius_cold"][i], flux_statistics["mom_flux_16"][i], flux_statistics["mom_flux_84"][i], None, coloring[i], "dashed", False)

    ax4.plot(e_radial_snapshots[i], 2*flux_statistics["eradial_flux_radius"][i]/P_flux_cgs[i]/UnitVelocity_in_cm_per_s, color=coloring[i], label=labeling[i], linestyle="solid")
    # ax1.plot(m_radial_snapshots[i], 2*flux_statistics["eradial_flux_radius_cold"][i]/P_flux_cgs_cold[i]/UnitVelocity_in_cm_per_s, color=coloring[i], linestyle="dashed")
    ax4.plot(m_radial_snapshots[i], flux_statistics["rad_vels"][i], color=coloring[i], linestyle="dashdot")



R_pc = R*1000
ax2.axvline(0.3, linestyle="dashed", color="black")
ax2.text(0.35, 3e43, r"$R_{\rm inj} = %0.0f$ pc" % R_pc, fontsize=14)
ax1.text(0.55, 18, r"$\Delta r = %0.2f$ kpc" % np.diff(m_radial_snapshots[0])[0], fontsize=14, backgroundcolor="white")
ax1.text(0.55, 16, r"Time Range = 48 - 50 Myr", fontsize=14, backgroundcolor="white")

plt.rcParams['legend.title_fontsize'] = 'large'
ax1.set_xlabel("Radius [kpc]", fontsize="x-large")
ax1.tick_params(axis='both', labelsize="large")
ax1.set_ylabel(r"$\rm \dot{M}$ [$\rm M_\odot \, yr^{-1}$]", fontsize="x-large")
ax1.set(xlim=(0.0, 20), ylim=(-5, 25))
l1 = ax1.legend(loc="upper right", fontsize=14)
l2 = ax1.legend(tcat_labels, tcat_names, loc="upper left", fontsize=14) #  title=r"$\rm \dot{M}$ vs r")
ax1.add_artist(l1)
ax1.add_artist(l2)


ax2.set_xlabel("Radius [kpc]", fontsize=14)
ax2.tick_params(axis='both', labelsize="large")
ax2.set_ylabel(r"$\rm \dot{E}$ [$\rm erg \, s^{-1}$]", fontsize="x-large")
ax2.set(ylim=(1e40, 1e45), xscale="log", yscale="log")
ax2.set(xlim=(0,20))
l1 = ax2.legend(loc="upper right", fontsize=14)
l3 = ax2.legend(em_labels, em_names, loc="upper left", fontsize=14) #  title=r"$\rm \dot{M}$ vs r")
ax2.add_artist(l1)
ax2.add_artist(l3)


# ax3.text(0.3, 16800, r"$\Delta r = %0.2f$ kpc" % np.diff(mom_radial_snapshots[0])[0], fontsize="x-large", backgroundcolor="white")
# ax3.text(0.3, 15500, r"Time Range = 48 - 50 Myr", fontsize="x-large", backgroundcolor="white")
ax3.set_xlabel("Radius [kpc]", fontsize=14)
ax3.tick_params(axis='both', labelsize="large")
ax3.set_ylabel(r"$\rm \dot{p}$ [$\rm M_\odot \, km\, s^{-1}\, yr^{-1}$]", fontsize="x-large")

ax3.set(xlim=(0.0, 20), ylim=(0, 22000))
l1 = ax3.legend(loc="upper right", fontsize=14)
l2 = ax3.legend(tcat_labels, tcat_names, loc="upper left", fontsize=14) #  title=r"$\rm \dot{M}$ vs r")
ax3.add_artist(l1)
ax3.add_artist(l2)


ax4.axvline(0.3, linestyle="dashed", color="black")
ax4.text(0.35, 200, r"$R_{\rm inj} = %0.0f$ pc" % R_pc, fontsize=14)


vnp_labels = [
                Line2D([0], [0], color='black', linestyle='solid'),        
                Line2D([0], [0], color='black', linestyle='dashdot'),  
            ]
vnp_names = ["Ratios", "Radial Velocities"] 

l1 = ax4.legend(vnp_labels, vnp_names, loc="upper left", fontsize=14)
l2 = ax4.legend(loc="upper right", fontsize=14)

ax4.tick_params(axis='both', labelsize="large")
ax4.set_xscale("log")
ax4.set_xlim(1e-1, 30)
ax4.set_ylim(0.0, 2500)
ax4.set_xlabel("Radius [kpc]", fontsize=14)
ax4.set_ylabel(r"$\rm \, 2\dot{E}/\dot{P}$ [km/s]", fontsize="x-large")

ax4.add_artist(l1)
ax4.add_artist(l2)

plt.savefig("mass_energy_outflow_rate_comparison.pdf", bbox_inches='tight', dpi=150)
plt.show()

### 2D Histogram of Temperature vs Radial Velocity, weighted by Outflow Rate and Radius

In [ ]:
coloring = color_map(np.linspace(0, 1, len(outputs)))

n_bins = 301
theta = 60
n_snaps = 1
r_analysis = [3] 
r_bins = np.linspace(0.3, 30, n_bins+1)
dr = np.diff(r_bins)[0]
t_bins = np.linspace(3.5, 8, n_bins+1)
nd_bins = np.linspace(-3, 5, n_bins+1)
rv_bins = np.linspace(0, 1800, n_bins+1)
rad_bins = np.linspace(0, 40, n_bins+1)

eps = boxsize/cells_per_dim

tv_flux = np.zeros(shape=(len(outputs), n_bins, n_bins))
rad_averages = np.zeros(shape=(len(outputs), n_bins, n_bins))
######

for o, output in enumerate(outputs):
    data = {}
    for i in np.arange(first, last + step, step):
        filename = output + "snap_%03d.hdf5" % i
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)
        boxsize = parameters["BoxSize"]
        alpha = parameters["E_load"]
        beta = parameters["M_load"]
        coord = data["Coordinates"]
        x_coord = data["Coordinates"][:,0] 
        y_coord = data["Coordinates"][:,1]
        z_coord = data["Coordinates"][:,2]
        density = data["Density"]
        number_density = density*UnitNumberDensity # Units: cm^{-3}

        internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
        masses = data["Masses"] 
        abundance = data["ElectronAbundance"]
        temperature = Temp_S(abundance, internal_energy)

        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1] 
        vel_z = data["Velocities"][:,2] 
        vel = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
        E = internal_energy*masses # NOTE: This is the actual internal energy
        R = parameters["injection_radius"]
        sfr = parameters["sfr"]
        eps = boxsize/cells_per_dim
        rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
        radius = np.sqrt(rad_x**2 + rad_y**2 + rad_z**2)
        radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s
        # tan_theta = rad_x/(rad_z + devx)
        # theta = np.arctan(tan_theta)*180/np.pi # this returns -90 to 90 
        theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
        bg_cells = (number_density <= nd_h) & (temperature >= t_l) & (np.abs(radial_velocity) <= 40) # Gets rids of most BG cells.

        angular_region = (np.abs(theta) <= angle_l) & np.abs(rad_z >= 0.3) & (radius <= 40) # Excludes anything with absolute angles greater than 60 
        angular_bg_mask = (angular_region) & (~bg_cells)
        ''' 
            According to TNG50, the figure that I'm basing this plot on different outflow phases in TNG presents the galaxy-stacked 2D histogram 
            of mass outflow rate, as a function of temperature (x-axis) and instantaneous velocity (y-axis).
        '''
        
        log10_temp = np.log10(temperature)
        log10_density = np.log10(number_density)
        log10_rv = np.log10(np.abs(radial_velocity))
        # H_2d, t_edge, rv_edge = np.histogram2d(log10_temp[angular_region], radial_velocity[angular_region], 
        #                            bins=(t_bins, rv_bins), weights=masses[angular_region]*radial_velocity[angular_region])

        H_2d, t_edge, rv_edge,_ = stats.binned_statistic_2d(x=log10_temp[angular_bg_mask], y=radial_velocity[angular_bg_mask], values=masses[angular_bg_mask]*radial_velocity[angular_bg_mask],
                                   bins=(t_bins, rv_bins), statistic="sum")
          

        r_2d, tr_edge, rvr_edge,_ = stats.binned_statistic_2d(x=log10_temp[angular_bg_mask], y=radial_velocity[angular_bg_mask], values=radius[angular_bg_mask],
                                   bins=(t_bins, rv_bins), statistic="mean")
        r_2d[np.isnan(r_2d)] = 0
        tv_flux[o] += H_2d/dr/(UnitTime_in_s/seconds_in_yrs)
        rad_averages[o] += r_2d # the main problem is I'm taking the average across a long period of time, if I have nan values then I can't take averages..

tv_flux_n = tv_flux/((last - first)/step + 1 ) 
rad_averages_n = rad_averages/ ((last - first)/step + 1)


In [ ]:

labels_cbar = [-3, -2, -1, 0, 1]
labels_v = [0, 500, 1000, 1500, 2000]
labels_T = [3.5, 5, 6.5, 8]

log10tv_flux = np.log10(tv_flux_n)
T, V = np.meshgrid(t_edge, rv_edge)
# outflow_mesh = ax0.pcolormesh(T, V, H_flux_0, shading='auto', cmap="plasma", vmin=0, vmax=4)
# cbar = plt.colorbar(outflow_mesh, ax = ax0, label=r"Outflow Rate [$M_\odot \, yr^{-1}$]") 
# ax0.set(xlabel=("Temperature [$log_{10}(K)$]"), ylabel=("Radial Velocity [$km/s$]"), ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(np.min(t_bins),np.max(t_bins)))
# plt.show()

fig = plt.figure(figsize=(10, 8))
fig.set_rasterized(True)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)
axs = [ax1, ax2, ax3, ax4]
for i, ax in enumerate(axs):
    TV_mesh =  ax.pcolormesh(T, V, log10tv_flux[i], shading='auto', cmap="mako", vmin=(-3, 1))
    ax.tick_params(axis='both', which='major', labelsize=11)

# outflow_mesh_1 = ax2.pcolormesh(T, V, log10tv_flux[1], shading='auto', cmap="mako", vmin=(-3, 1))
# outflow_mesh_2 = ax3.pcolormesh(T, V, log10tv_flux[2], shading='auto', cmap="mako", vmin=(-3, 1))
# outflow_mesh_3 = ax4.pcolormesh(T, V, log10tv_flux[3], shading='auto', cmap="mako", vmin=(-3, 1))

    # ax.tick_params(axis='both', which='major', labelsize=11)

ax1.set(ylim=(np.min(rv_bins),2000), xlim=(3.5, 8))
ax1.set_ylabel(ylabel=(r"Radial Velocity [$\rm km/s$]"), fontsize=12)
ax2.set(ylim=(np.min(rv_bins),2000), xlim=(3.5, 8))
ax3.set(ylim=(np.min(rv_bins), 2000), xlim=(3.5, 8))
ax3.set_ylabel(ylabel=(r"Radial Velocity [$\rm km/s$]"), fontsize=12)
ax3.set_xlabel(xlabel=(r"Temperature [$\rm log_{10}(K)$]"), fontsize=12)
ax4.set(ylim=(np.min(rv_bins), 2000), xlim=(3.5, 8))
ax4.set_xlabel(xlabel=(r"Temperature [$\rm log_{10}(K)$]"), fontsize=12)
ax1.text(3.8, 1800, labeling[0], fontsize=13, backgroundcolor="white")
ax2.text(3.8, 1800, labeling[1], fontsize=13, backgroundcolor="white")
ax3.text(3.8, 1800, labeling[2], fontsize=13, backgroundcolor="white")
ax4.text(3.8, 1800, labeling[3], fontsize=13, backgroundcolor="white")
ax1.text(3.8, 1600, r"Time Range = 48 - 50 Myr", fontsize=13, backgroundcolor="white")

fig.subplots_adjust(right=0.84, wspace = 0.1, hspace=0.1)
cbar_ax = fig.add_axes([0.86, 0.125, 0.025, 0.755])
cbar_mass_outflow = fig.colorbar(TV_mesh, cax=cbar_ax)
cbar_mass_outflow.set_ticks(labels_cbar)
cbar_mass_outflow.set_ticklabels(labels_cbar)
cbar_mass_outflow.set_label(r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]", fontsize=12)
# cbar_mass_outflow = plt.colorbar(outflow_mesh_3, ax = ax4, label=r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]") 
plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.setp(ax4.get_yticklabels(), visible=False)
# ax1.set_yticks(labels_v)
# ax3.set_yticks(labels_v)
# ax3.set_xticks(labels_T)
# ax4.set_xticks(labels_T)

plt.savefig("temperature_rv_phase_load.pdf", bbox_inches='tight', dpi=150)

plt.show()


In [ ]:

fig = plt.figure(figsize=(7, 5))
# ax0 = fig.add_subplot(111)
Tr, Vr = np.meshgrid(tr_edge, rvr_edge)
print(Tr.all() == T.all())
# outflow_mesh = ax0.pcolormesh(T, V, H_flux_0, shading='auto', cmap="plasma", vmin=0, vmax=4)
# cbar = plt.colorbar(outflow_mesh, ax = ax0, label=r"Outflow Rate [$M_\odot \, yr^{-1}$]") 
# ax0.set(xlabel=("Temperature [$log_{10}(K)$]"), ylabel=("Radial Velocity [$km/s$]"), ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(np.min(t_bins),np.max(t_bins)))
# plt.show()

fig = plt.figure(figsize=(10, 8))
fig.set_rasterized(True)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

rad_averages_f = rad_averages_n
rad_averages_f[rad_averages_n == 0] = np.nan

outflow_mesh_fid = ax1.pcolormesh(Tr, Vr, rad_averages_f[0], shading='auto', cmap="inferno", vmin=(0, 40))
outflow_mesh_1 = ax2.pcolormesh(Tr, Vr, rad_averages_f[1], shading='auto', cmap="inferno", vmin=(0, 40))
outflow_mesh_2 = ax3.pcolormesh(Tr, Vr, rad_averages_f[2], shading='auto', cmap="inferno", vmin=(0, 40))
outflow_mesh_3 = ax4.pcolormesh(Tr, Vr, rad_averages_f[3], shading='auto', cmap="inferno", vmin=(0, 40))

ax1.set(ylim=(np.min(rv_bins),2000), xlim=(3.5, 8))
ax1.set_ylabel(ylabel=(r"Radial Velocity [$\rm km/s$]"), fontsize=12)
ax2.set(ylim=(np.min(rv_bins),2000), xlim=(3.5, 8))
ax3.set(ylim=(np.min(rv_bins), 2000), xlim=(3.5, 8))
ax3.set_ylabel(ylabel=(r"Radial Velocity [$\rm km/s$]"), fontsize=12)
ax3.set_xlabel(xlabel=(r"Temperature [$\rm log_{10}(K)$]"), fontsize=12)
ax4.set(ylim=(np.min(rv_bins), 2000), xlim=(3.5, 8))
ax4.set_xlabel(xlabel=(r"Temperature [$\rm log_{10}(K)$]"), fontsize=12)
ax1.text(3.8, 1800, labeling[0], fontsize=13, backgroundcolor="white")
ax2.text(3.8, 1800, labeling[1], fontsize=13, backgroundcolor="white")
ax3.text(3.8, 1800, labeling[2], fontsize=13, backgroundcolor="white")
ax4.text(3.8, 1800, labeling[3], fontsize=13, backgroundcolor="white")
ax1.text(3.8, 1600, r"Time Range = 48 - 50 Myr", fontsize=13, backgroundcolor="white")



fig.subplots_adjust(right=0.84, wspace = 0.1, hspace=0.1)
cbar_ax = fig.add_axes([0.86, 0.125, 0.018, 0.755])
cbar_mass_outflow = fig.colorbar(outflow_mesh_3, cax=cbar_ax)
# cbar_mass_outflow.set_ticks([0, 1, 2])
# cbar_mass_outflow.set_ticklabels(labels_cbar)
cbar_mass_outflow.set_label(r"Radius [kpc]", fontsize=12)
# cbar_mass_outflow = plt.colorbar(outflow_mesh_3, ax = ax4, label=r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]") 
plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.setp(ax4.get_yticklabels(), visible=False)
ax1.set_yticks(labels_v)
ax3.set_yticks(labels_v)
ax3.set_xticks(labels_T)

plt.savefig("temperature_rv_distance_load.pdf", bbox_inches='tight', dpi=150)

plt.show()

### 2D Histogram of Radial Velocity vs Density, weighted by Outflow Rate and Radius

In [ ]:
coloring = color_map(np.linspace(0, 1, len(outputs)))

n_bins = 301
theta = 60
n_snaps = 1
r_analysis = [3] 
r_bins = np.linspace(0.3, 30, n_bins+1)
dr = np.diff(r_bins)[0]
t_bins = np.linspace(3.5, 8, n_bins+1)
nd_bins = np.linspace(-4, 2, n_bins+1)
rv_bins = np.linspace(0, 1800, n_bins+1)
rad_bins = np.linspace(0, 40, n_bins+1)

eps = boxsize/cells_per_dim
tn_flux = np.zeros(shape=(len(outputs), n_bins, n_bins))
rad_averages = np.zeros(shape=(len(outputs), n_bins, n_bins))

######

for o, output in enumerate(outputs):
    data = {}
    for i in np.arange(first, last + step, step):
        filename = output + "snap_%03d.hdf5" % i
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)
        boxsize = parameters["BoxSize"]
        alpha = parameters["E_load"]
        beta = parameters["M_load"]
        coord = data["Coordinates"]
        x_coord = data["Coordinates"][:,0] 
        y_coord = data["Coordinates"][:,1]
        z_coord = data["Coordinates"][:,2]
        density = data["Density"]
        number_density = density*UnitNumberDensity # Units: cm^{-3}

        internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
        masses = data["Masses"] 
        abundance = data["ElectronAbundance"]
        temperature = Temp_S(abundance, internal_energy)

        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1] 
        vel_z = data["Velocities"][:,2] 
        vel = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
        E = internal_energy*masses # NOTE: This is the actual internal energy
        R = parameters["injection_radius"]
        sfr = parameters["sfr"]
        eps = boxsize/cells_per_dim
        rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
        radius = np.sqrt(rad_x**2 + rad_y**2 + rad_z**2)
        radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s
        # tan_theta = rad_x/(rad_z + devx)
        # theta = np.arctan(tan_theta)*180/np.pi # this returns -90 to 90 
        theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
        bg_cells = (number_density <= nd_h) & (temperature >= t_l) & (np.abs(radial_velocity) <= 50) # Gets rids of most BG cells.

        angular_region = (np.abs(theta) <= angle_l) & np.abs(rad_z >= 0.3) & (radius <= 40) # Excludes anything with absolute angles greater than 60 
        angular_bg_mask = (angular_region) & (~bg_cells)
        ''' 
            According to TNG50, the figure that I'm basin this plot on different outflow phases in TNG presents the galaxy-stacked 2D histogram 
            of mass outflow rate, as a function of temperature (x-axis) and instantaneous velocity (y-axis).
        '''
        
        log10_temp = np.log10(temperature)
        log10_density = np.log10(number_density)
        log10_rv = np.log10(np.abs(radial_velocity))
        # H_2d, t_edge, rv_edge = np.histogram2d(log10_temp[angular_region], radial_velocity[angular_region], 
        #                            bins=(t_bins, rv_bins), weights=masses[angular_region]*radial_velocity[angular_region])

        H_2d, nd_edge, rv_edge,_ = stats.binned_statistic_2d(x=log10_density[angular_bg_mask], y=radial_velocity[angular_bg_mask], values=masses[angular_bg_mask]*radial_velocity[angular_bg_mask],
                                   bins=(nd_bins, rv_bins), statistic="sum")
          
        r_2d, ndr_edge, rvr_edge,_ = stats.binned_statistic_2d(x=log10_density[angular_bg_mask], y=radial_velocity[angular_bg_mask], values=radius[angular_bg_mask],
                                   bins=(nd_bins, rv_bins), statistic="mean")

        r_2d[np.isnan(r_2d)] = 0
        tn_flux[o] += H_2d/dr/(UnitTime_in_s/seconds_in_yrs)
        rad_averages[o] += r_2d # the main problem is I'm taking the average across a long period of time, if I have nan values then I can't take averages..

tn_flux_n = tn_flux/((last - first)/step + 1 ) 
rad_averages_n = rad_averages/ ((last - first)/step + 1)


In [ ]:

labels_cbar = [-3, -2, -1, 0, 1]
labels_v = [0, 500, 1000, 1500, 2000]
labels_T = [3.5, 5, 6.5, 8]

log10tn_flux = np.log10(tn_flux)
T, N = np.meshgrid(nd_edge, rv_edge)
# outflow_mesh = ax0.pcolormesh(T, V, H_flux_0, shading='auto', cmap="plasma", vmin=0, vmax=4)
# cbar = plt.colorbar(outflow_mesh, ax = ax0, label=r"Outflow Rate [$M_\odot \, yr^{-1}$]") 
# ax0.set(xlabel=("Temperature [$log_{10}(K)$]"), ylabel=("Radial Velocity [$km/s$]"), ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(np.min(t_bins),np.max(t_bins)))
# plt.show()

fig = plt.figure(figsize=(10, 8))
fig.set_rasterized(True)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

axs = [ax1, ax2, ax3, ax4]
for i, ax in enumerate(axs):
    T_nd_hist = ax.pcolormesh(T, N, log10tn_flux[i], shading='auto', cmap="mako", vmin=(-3, 1))
    ax.tick_params(axis='both', which='major', labelsize=11)

# outflow_mesh_1 = ax2.pcolormesh(T, V, log10tv_flux[1], shading='auto', cmap="mako", vmin=(-3, 1))
# outflow_mesh_2 = ax3.pcolormesh(T, V, log10tv_flux[2], shading='auto', cmap="mako", vmin=(-3, 1))
# outflow_mesh_3 = ax4.pcolormesh(T, V, log10tv_flux[3], shading='auto', cmap="mako", vmin=(-3, 1))

ax1.set(ylim=(np.min(rv_bins),2000), xlim=(-4, 2))
ax1.set_ylabel(ylabel=(r"Radial Velocity [$\rm km/s$]"), fontsize=12)
ax2.set(ylim=(np.min(rv_bins),2000), xlim=(-4, 2))

ax3.set(ylim=(np.min(rv_bins), 2000), xlim=(-4, 2))
ax3.set_ylabel(ylabel=(r"Radial Velocity [$\rm km/s$]"), fontsize=12)
ax3.set_xlabel(xlabel=(r"Density [$\rm log_{10}(cm^{-3})$]"), fontsize=12)
ax4.set(ylim=(np.min(rv_bins), 2000), xlim=(-4, 2))
ax4.set_xlabel(xlabel=(r"Density [$\rm log_{10}(cm^{-3})$]"), fontsize=12)

ax1.text(-3.5, 1800, labeling[0], fontsize=13, backgroundcolor="white")
ax2.text(-3.5, 1800, labeling[1], fontsize=13, backgroundcolor="white")
ax3.text(-3.5, 1800, labeling[2], fontsize=13, backgroundcolor="white")
ax4.text(-3.5, 1800, labeling[3], fontsize=13, backgroundcolor="white")
ax1.text(-3.5, 1600, r"Time Range = 48 - 50 Myr", fontsize=13, backgroundcolor="white")


# cbar1 = plt.colorbar(outflow_mesh_fid, ax = ax1, label=r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]") 
# cbar2 = plt.colorbar(outflow_mesh_1, ax = ax2, label=r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]") 
# cbar3 = plt.colorbar(outflow_mesh_2, ax = ax3, label=r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]") 
# cbar3 = plt.colorbar(outflow_mesh_3, ax = ax4, label=r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]") 


fig.subplots_adjust(right=0.84, wspace = 0.1, hspace=0.1)
cbar_ax = fig.add_axes([0.86, 0.125, 0.025, 0.755])
cbar_mass_outflow = fig.colorbar(T_nd_hist, cax=cbar_ax)
cbar_mass_outflow.set_ticks(labels_cbar)
cbar_mass_outflow.set_ticklabels(labels_cbar)
cbar_mass_outflow.set_label(r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]", fontsize=12)
# cbar_mass_outflow = plt.colorbar(outflow_mesh_3, ax = ax4, label=r"Outflow Rate [$\rm M_\odot \, yr^{-1}$]") 
plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.setp(ax4.get_yticklabels(), visible=False)

plt.savefig("density_rv_phase.pdf", bbox_inches='tight', dpi=150)

plt.show()


In [ ]:

    # r_2d, r_edge, rvr_edge,_ = stats.binned_statistic_2d(x=log10_density[angular_bg_mask], y=radial_velocity[angular_bg_mask], values=radius[angular_bg_mask],
    #                                bins=(nd_bins, rv_bins), statistic="mean")
          
        # rad_averages[o] += r_2d

# tn_flux_n = tn_flux/((last - first)/step + 1 ) 
# rad_averages_n = rad_averages/ ((last - first)/step + 1)

# fig = plt.figure(figsize=(7, 5))
# ax0 = fig.add_subplot(111)
T, N = np.meshgrid(ndr_edge, rvr_edge)
# outflow_mesh = ax0.pcolormesh(T, V, H_flux_0, shading='auto', cmap="plasma", vmin=0, vmax=4)
# cbar = plt.colorbar(outflow_mesh, ax = ax0, label=r"Outflow Rate [$M_\odot \, yr^{-1}$]") 
# ax0.set(xlabel=("Temperature [$log_{10}(K)$]"), ylabel=("Radial Velocity [$km/s$]"), ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(np.min(t_bins),np.max(t_bins)))
# plt.show()

fig = plt.figure(figsize=(10, 8))
fig.set_rasterized(True)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

rad_averages_f = rad_averages_n
rad_averages_f[rad_averages_n == 0] = np.nan

outflow_mesh_fid = ax1.pcolormesh(T, N, rad_averages_f[0], shading='auto', cmap="inferno", vmin=(0, 40))
outflow_mesh_1 = ax2.pcolormesh(T, N, rad_averages_f[1], shading='auto', cmap="inferno", vmin=(0, 40))
outflow_mesh_2 = ax3.pcolormesh(T, N, rad_averages_f[2], shading='auto', cmap="inferno", vmin=(0, 40))
outflow_mesh_3 = ax4.pcolormesh(T, N, rad_averages_f[3], shading='auto', cmap="inferno", vmin=(0, 40))

ax1.set(ylabel=("Radial Velocity [$km/s$]"),  ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(-4, 2))
ax2.set(ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(-4, 2))
ax3.set(xlabel=("Density [$log_{10}(cm^{-3})$]"), ylabel=("Radial Velocity [$km/s$]"),  ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(-4, 2))
ax4.set(xlabel=("Density [$log_{10}(cm^{-3})$]"), ylim=(np.min(rv_bins),np.max(rv_bins)), xlim=(-4, 2))
ax1.text(-3.5, 2000, labeling[0], fontsize=13, backgroundcolor="white")
ax2.text(-3.5, 2000, labeling[1], fontsize=13, backgroundcolor="white")
ax3.text(-3.5, 2000, labeling[2], fontsize=13, backgroundcolor="white")
ax4.text(-3.5, 2000, labeling[3], fontsize=13, backgroundcolor="white")
ax1.text(-3.5, 2000, r"Time Range = 48 - 50 Myr", fontsize=13, backgroundcolor="white")



cbar1 = plt.colorbar(outflow_mesh_fid, ax = ax1, label=r"Distance [kpc]") 
cbar2 = plt.colorbar(outflow_mesh_1, ax = ax2, label=r"Distance [kpc]") 
cbar3 = plt.colorbar(outflow_mesh_2, ax = ax3, label=r"Distance [kpc]") 
cbar3 = plt.colorbar(outflow_mesh_3, ax = ax4, label=r"Distance [kpc]") 

plt.savefig("density_rv_distance_load.pdf", bbox_inches='tight', dpi=150)

plt.show()